# Super Smash Bros Ultimate 

In [300]:
# This Project extracts data from a website 'https://ultimateframedata.com/'
# The code is design so that the user can run all 'cells' from top to bottom.
# But if the user wants only offensive data or non-offensive they select those specfic cells after
# selecting all the other cells.

In [301]:
def get_tables (source, offensive):
    table_name = []
    if offensive:
        temp = source.find_all('h2')
        for i in range( len(temp)):
            temp[i] = temp[i].get_text()
            temp[i] = clean_up(temp[i])
            if temp[i].find('/') > -1:
                temp[i] = temp[i].replace('/', 'and')
            table_name.append(temp[i])
            
    else:
        for i in range( len(source)):
            temp = source[i]
            temp = temp.find('h2').get_text()
            table_name.append(temp)
    
    return table_name

In [302]:
# cleans up the str by removing unnessary characters
def clean_up(str):
    to_del = ['\r','\n','\t','\\','[',']','\'','\"', '\xa0']
    for i in range(len(to_del)):
        str = str.replace(to_del[i],'')
    return str

In [303]:
# creates and return list for columns port of our dataframe(s)


def get_col (source, offensive, table_num):
    col = []
    movecontainer = source.find('div')
    temp = movecontainer.find_all('div')
    if offensive:
        if table_num != 6:
            
            
            for i in range(len(temp)):
                temp[i] = temp[i].get('class')
                temp[i] = str(temp[i])
                temp[i] = clean_up(temp[i])
                if temp[i] != 'hitbox':
                    col.append(temp[i])
    else:
        th = source.find_all('th')
        
        for i in range(len(th)):
            th[i] = th[i].get_text()
            th[i] = clean_up(th[i])
            th[i] = (th[i]).strip()
            #if th[i] != 'Rank' and th[i].isdigit() == False:
            if th[i].isdigit() :
                break
            col.append(th[i])
    return col


In [304]:
# creates and return list for rows port of our dataframe(s)

def get_row(source, offensive, table_num, col_len):

    row = []
    temp = source.find_all('div')
    if offensive:
        if table_num != 6:
            
            
            for i in range(len(temp)):
                temp[i] = temp[i].get_text()
                temp[i] = clean_up(temp[i])
                if temp[i] != 'hitbox' and len(temp[i]) > 0:
                    row.append(temp[i])
        if len(row) - 1 == col_len:
            if ((row[0].isdigit() == False ) and (row[1].isdigit() == False)):
                del row[0]
        if row[0] == 'Pummel' and len(row) == col_len - 2:
            row.append('NULL')
            row.append('NULL')
        elif row[0].find(' Throw') > -1 and len(row) == col_len - 2:
            row.append('NULL')
            row.append('NULL')
            
        return row
    else :
        tr = source.find_all('tr')
        
        for i in range(len(tr)):
            td = tr[i].find_all('td')
            
            for j in range(len(td)) :
                td[j] = td[j].get_text()
                td[j] = clean_up(td[j])
                
            row.append(td)
    return row


In [305]:
def get_col_and_row (moves):
    row = []
    col = []
    div = moves.find_all('div')
    len_div = len(div)
    for i in range(2,len_div):
        line = div[i].get_text()
        line = clean_up(line)
        if line.find('— ') == -1:
            
            break
        index = line.index('— ')
        sub_col = line[0:index]
        sub_col = sub_col.strip()
        sub_row = line[index + 2:]
        sub_row = sub_row.strip()
        col.append(sub_col)
        row.append(sub_row)
    return col , row

In [306]:
def get_links():
    df = non_off_df[0]
    characters = (df['Character' ].to_list())
    url = 'https://ultimateframedata.com/'
    print(characters[0])
    link = []
    for c in range(len(characters)):
        suffix = characters[c]
        suffix = suffix.replace(' ','_')
        suffix = suffix.replace('-','_')
        suffix = suffix.replace('.','')
        suffix = suffix.replace('&','and')
        suffix = suffix.lower()
        if suffix == 'rosalina':
            suffix = 'rosalina_and_luma'
        if suffix == 'dedede':
            suffix = 'king_dedede'
        link.append( url + suffix )
    return link, characters

# Non-Offensive Data

In [307]:
# consider this block the table of contents of extraction port of this project
# this block will be broken to parts. non-offensive stats like weight and speed, and offensive stats like 
import os 
import shutil as sh
from bs4 import BeautifulSoup as be
import pandas as pd
import requests
import re
import matplotlib

##################################
#non-offensive stats extraction, formating in to a dataframe

url = 'https://ultimateframedata.com/stats'
page = requests.get(url)
soup = be(page.text, 'html')

page_source = soup.find('div', class_= 'statspagecontainer')

source = page_source.find_all('div', class_='statstablecontainer')

number_of_characters = 85
table_names = get_tables(source, False)
number_of_tables = len(table_names)


non_off_df = []

for i in range(number_of_tables):
    col = get_col(source[i],False, -i -1)
    
    non_off_df.append( pd.DataFrame(columns= col) )

    list_of_rows = get_row(source[i], False, len(col), -1)
    if len(list_of_rows[1]) !=  len(col):
        if len(list_of_rows[1]) <  len(col):
            del col[0]
    
    for j in range(1, len(list_of_rows)):

        # Skips the the table 'Reflectors' Because most character don't have this ability
        if i != 17:
            non_off_df[i].loc[j] = list_of_rows[j]
        
            
to_excel(' ', table_names, non_off_df, False)            
#len(non_off_df)
print('Done')

DataFrames have been successfully exported to D:\career\Python\ssb\ssb non offensive\non_offensive.xlsx
Done


# Offensive Data

In [309]:

#offensive stats extraction, formating in to a dataframe
# offensive stats are stored the character's own page 

offensive_df = []


character_link = []
character = []
character_link, character  = get_links()
c_len =  len(character_link)

# loops through each character
for i in range(c_len ):
    # Each character's attack types will be repersented as an index in this list. 
    # Each index will be a dataframes containing attack stats as column, and the values of that stat 
    # Each iteration of the 'i' index will clear this list
    
    off_df = []
    off_df.clear()

    
    
    # these characters fall under the name "Ice Climbers"
    if character[i] in  ['Nana' , 'Popo']:
        continue
    page = requests.get(character_link[i] )
    soup = be(page.text, 'html')
    page_source = soup.find('div', id= 'contentcontainer')
    
    # Gets all tables applicable for this character
    off_table_names = []
    off_table_names.clear()
    off_table_names = get_tables(page_source, True)
    
    len_off_table_names = len(off_table_names)


    # contains all the data for a category moves (i.e. grabs, Ground attacks)
    moves = page_source.find_all('div', class_='moves')
    
    
    # for this specific table
    for j in range( len_off_table_names):

        if j < 5:
            off_col = []
            off_col.clear()
            off_col = get_col(moves[j], True,j)
            
            off_df.append(pd.DataFrame(columns = off_col))
            
            move_details = moves[j].find_all('div', class_='movecontainer')
            
            off_row = []
            off_row.clear()
            for k in range( len(move_details) ):
                
                off_row = get_row(move_details[k], True,j, len(off_col)  )

                while len(off_row) > len(off_col):
                    del off_row[0]
                
                while len(off_row) < len(off_col):
                    off_row.append('Null')
                          
                    
                off_df[j].loc[k] = off_row

                
        else:
            # misc table actually stores the column data in same the way it stores the row data
            off_col, off_row = get_col_and_row(moves[j])
            off_df.append(pd.DataFrame(columns = off_col))
            off_df[-1].loc[-1] = off_row
    
    offensive_df.append(off_df)         
    to_excel(character[i], off_table_names, off_df, True)

print(len(offensive_df))

Jigglypuff
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Jigglypuff.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Wario.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Peach.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Daisy.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Palutena.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Rosalina.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Mega Man.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Squirtle.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Mr. Game & Watch.xlsx
DataFrames have been successfully exported to D:\career\Python\ssb\ssb_offensive\Pichu.xlsx
DataFrames have been successfully exporte

# Exporting methods

In [311]:
# Output file name for offensive dataframes
def to_excel(character, table_names, df, offensive):
    if offensive:
        output_name = "D:\\career\\Python\\ssb\\ssb_offensive\\"
        output_name = output_name + character + ".xlsx"
    else:
        output_name = r"D:\career\Python\ssb\ssb non offensive\non_offensive.xlsx"

    # Create an Excel writer object
    with pd.ExcelWriter(output_name) as writer:
        for dataframe, sheet_name in zip(df, table_names):
            dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"DataFrames have been successfully exported to {output_name}")